# Generalized QSP

In [ ]:
import numpy as np
import scipy
from qualtran.drawing import show_bloq

import qualtran
from qualtran.bloqs.qubitization_walk_operator_test import get_walk_operator_for_1d_Ising_model
from qualtran.bloqs.qubitization_walk_operator import QubitizationWalkOperator
from qualtran.cirq_interop import CirqGateAsBloq

from qualtran.bloqs.generalized_qsp import GeneralizedQSP, HamiltonianSimulationByGQSP
from qualtran.bloqs.generalized_qsp_test import RandomGate

`GeneralizedQSP` implements the Quantum Eigenvalue Transform on a unitary $U$ using QSP. Given a complex GQSP polynomial $P$ (and its complement $Q$), it implements the unitary:
$$U' = \begin{bmatrix} P(U) & \cdot \\ Q(U) & \cdot \end{bmatrix}$$

Here, the polynomials $P, Q$ must satisfy the following constraint:

$$\left| P(e^{i\theta}) \right|^2 + \left| Q(e^{i\theta}) \right|^2 = 1 ~~\text{for every}~ \theta \in [0, 2\pi]$$

A polynomial $P$ is said to be a GQSP polynomial iff it satisfies $\left| P(e^{i\theta}) \right|^2 \le 1$ for every $\theta \in [0, 2\pi]$. 

Reference: https://doi.org/10.48550/arXiv.2308.01501

In [ ]:
U = get_walk_operator_for_1d_Ising_model(4, 2e-1)
show_bloq(U.decompose_bloq())

In [ ]:
pU = GeneralizedQSP(U, (0.5, 0.5), (-0.5, 0.5))
show_bloq(pU.decompose_bloq())

There is also a method that directly computes $Q$ from $P$:

In [ ]:
pU = GeneralizedQSP.from_qsp_polynomial(U, (0.5, 0, 0.5))
show_bloq(pU.decompose_bloq())

### Negative degree terms

To apply GQSP for a polynomial $P'(z) = z^{-k} P(z)$, we can just pass the polynomial $P$ along with negative power $k$.
The QSP angle sequence is the same for both, and $P'$ can be achieved by running $(U^\dagger)^k$ at any point in the circuit.

In [ ]:
pU = GeneralizedQSP.from_qsp_polynomial(U, (0.5, 0, 0.5), negative_power=1)
show_bloq(pU.decompose_bloq())

## Hamiltonian Simulation by GQSP

Given the Szegedy Quantum Walk Operator for a Hamiltonian $H$, one can construct the walk operator for $e^{-iHt}$ using GQSP (Corollary 8).

### Recap:

For a Hamiltonian $H = \sum_i \alpha_i U_i$, given the SELECT and PREPARE oracles
$$ \text{SELECT} = \sum_j \ketbra{j}{j} \otimes U_j $$
$$ \text{PREPARE} \ket{0} = \sum_j \frac{\sqrt{\alpha_j}}{\|\alpha\|_1} \ket{j} $$
we can implement the [QubitizationWalkOperator](qubitization_walk_operator.ipynb) that encodes the spectrum of $H$ in the eigenphases of the walk operator $W$.

### Approximating $\cos$
We can use the Jacobi-Anger expansion to obtain low-degree polynomial approximations for the $\cos$ function:

$$e^{it\cos\theta} = \sum_{n = -\infty}^{\infty} i^n J_n(t) (e^{i\theta})^n$$

where $J_n$ is the $n$-th [Bessel function of the first kind](https://en.wikipedia.org/wiki/Bessel_function#Bessel_functions_of_the_first_kind), which is provided by `scipy.special.jv`.

We can cutoff at $d = O(t + \log(1/\epsilon) / \log\log(1/\epsilon))$ to get an $\epsilon$-approximation (Theorem 7):

$$P[t](z) = \sum_{n = -d}^d i^n J_n(t) z^n$$

### Obtaining $e^{iHt}$

As the eigenphases of the walk operator are $e^{-i\arccos(E_k / \|\alpha\|_1)}$, we can use the GQSP polynomial with $P = P[\|\alpha\|_1 t]$ (and complementary $Q = 0$) to obtain $P(U) = e^{-iHt}$.
The obtained GQSP operator $W'$ can then be used with the PREPARE oracle to simulate the hamiltonian:
$$(\langle0| \otimes \text{PREPARE}^\dagger \otimes I) W' (|0\rangle \otimes \text{PREPARE} \otimes I) |0\rangle|\psi\rangle = |0\rangle e^{-iHt}|\psi\rangle$$

In [ ]:
W_e_iHt = HamiltonianSimulationByGQSP(U, t=5, alpha=1, precision=1e-3)

def decomp_pred(inst):
    from qualtran.bloqs.select_and_prepare import PrepareOracle, SelectOracle

    return any(isinstance(inst.bloq, cls) for cls in [HamiltonianSimulationByGQSP, GeneralizedQSP, SelectOracle, PrepareOracle])
    
show_bloq(
    W_e_iHt
    .decompose_bloq()
    .flatten(pred=decomp_pred)
)

In [ ]:
W_e_iHt.call_graph()